In [ ]:
import os
import shutil

In [ ]:
# !wget -O news.csv https://www.dropbox.com/s/uwa7c5uz7cac7cw/VMMRdb.zip?dl=0
# os.mkdir("data")
# !unzip news.csv -d data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
from tqdm import tqdm_notebook as tn
import json
from torch.utils.data import Dataset, DataLoader

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import random
import numpy as np
import torch.backends.cudnn as cudnn

def set_random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    random.seed(seed)
    np.random.seed(seed)
    
def prepare_cudnn(deterministic=True, benchmark=False):
    if torch.cuda.is_available():
        # CuDNN reproducibility
        # https://pytorch.org/docs/stable/notes/randomness.html#cudnn
        cudnn.deterministic = deterministic

        # https://discuss.pytorch.org/t/how-should-i-disable-using-cudnn-in-my-code/38053/4
        cudnn.benchmark = benchmark
        
def set_deterministic_behaviour(seed=42):
    set_random_seed(seed)
    prepare_cudnn(deterministic=True, benchmark=False)

In [ ]:
carsList = []
modelList = []
for carMarkInfo in carsInfo["list"]:
  mark = carMarkInfo["title"].lower()
  carsList.append(mark)
  for modelInfo in carMarkInfo["models"]:
    model = modelInfo["title"].lower()
    modelList.extend(model.strip().split(" "))

In [ ]:
class CarDataset(Dataset):
    
    def __init__(self, transforms):
      self.standfordCarsFolder =..
      self.carsFolder = ...
      self.transforms = transforms
      self.classes = set()
      self.classesMap = dict()

      self.cars = []
      for model in os.listdir(self.standfordCarsFolder):
        pathModel = os.path.join(self.standfordCarsFolder, model)
        carModel = "_".join(model.lower().strip().split(" "))

        if int(carModel.split("_")[-1]) < 2000:
          continue

        has = False
        for carModelFromList in carsList:
          if carModelFromList in carModel:
            has = True
            break
        if not has:
          continue
        has = False
        for carModelFromList in modelList:
          if carModelFromList in carModel:
            has = True
            break
        if not has:
          continue

        self.classes.add(carModel)
        self.classesMap[carModel] = len(self.classes)
        

        for carImage in os.listdir(pathModel):
          self.cars.append([carModel, os.path.join(pathModel, carImage)])

      for model in os.listdir(self.carsFolder):
        if int(model.split("_")[-1]) < 2000:
          continue
        pathModel = os.path.join(self.carsFolder, model)

        has = False
        for carModelFromList in carsList:
          if carModelFromList in model:
            has = True
            break
        if not has:
          continue
        has = False
        for carModelFromList in modelList:
          if carModelFromList in carModel:
            has = True
            break
        if not has:
          continue

        self.classes.add(model)
        self.classesMap[model] = len(model)
        
        for carImage in os.listdir(pathModel):
          self.cars.append([model, os.path.join(pathModel, carImage)])
      
      self.classes = list(self.classes)


    def __len__(self):
        return len(self.cars)


    def __getitem__(self, idx):
      image = self.transforms(Image.open(self.cars[idx][1]).convert('RGB'))
      
      label = self.classesMap[self.cars[idx][0]]

      return image, label, self.cars[idx][1]



In [ ]:
dataset = CarDataset(transforms=transforms.Compose([
                                              transforms.Resize((224,224)),
                                               transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225]),
                                               ] ))

In [ ]:
def load_image(pathToImage):
  try:
    pilImage = Image.open(pathToImage)
    pilImage = pilImage.convert('RGB')
    return pilImage
  except:
    imRaw = pyheif.read(pathToImage)
    pilImage = Image.frombytes(mode=imRaw.mode, size=imRaw.size, data=imRaw.data)
    pilImage = pilImage.convert('RGB')
    return pilImage
    
class CarDataset2(Dataset):
  
  def __init__(self, transforms):
    self.carFolder = "./BTBCars/"
    
    self.transforms = transforms
    self.classes = set()
    self.classesMap = dict()

    self.cars = []
    for model in os.listdir(self.carFolder):
      pathModel = os.path.join(self.carFolder, model)
      carModel = "_".join(model.lower().strip().split(" "))

      self.classes.add(carModel)
      self.classesMap[carModel] = len(self.classes) + len(dataset.classes)
      

      for carImage in os.listdir(pathModel):
        self.cars.append([carModel, os.path.join(pathModel, carImage)])

    self.classes = list(self.classes)


  def __len__(self):
      return len(self.cars)


  def __getitem__(self, idx):
    image = self.transforms(load_image(self.cars[idx][1]).convert('RGB'))
    
    label = self.classesMap[self.cars[idx][0]]

    return image, label, self.cars[idx][1]

from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

!pip install pyheif
import pyheif
dataset2 = CarDataset2(transforms=transforms.Compose([
                                              transforms.Resize((224,224)),
                                             #  transforms.RandomCrop(224,224),
                                              #  transforms.Resize((1300,1300)),
                                             #   transforms.RandomCrop(1024,1024),
                                               transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225]),
                                               ] )
                      )
batch_size = 64
set_deterministic_behaviour()
data_loader2 = torch.utils.data.DataLoader(dataset2,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
batch_size = 64
set_deterministic_behaviour()
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
import torchvision.models as models

In [ ]:
model = nn.Sequential( *list(models.resnet50(pretrained=True).children())[:-1],\
                      nn.Flatten(1),
                      nn.Linear(in_features=2048, out_features=len(dataset.classes), bias=True)
                     )


In [ ]:
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
epoch_num = 15

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
folderToDump = ...
try:
  os.mkdir(folderToDump)
except:
  pass

In [ ]:
def train(train_dataloader, net, optimizer, loss_func, epoch_num, w_decay=0):
    set_deterministic_behaviour()
    net.cuda()

    lr = 0.0001
    test_losses = []
    train_losses = []
    test_acces = []
    for epoch in range(epoch_num):
        if epoch in [0, 7, 11]:
            lr = lr / 10
            optimizer = torch.optim.Adam(net.parameters(), weight_decay=w_decay, lr=lr)

        net.train()
        train_loss = []
        progress = tn(train_dataloader)
        for x, y in progress:
            x = x.cuda()
            y = y.cuda()
            optimizer.zero_grad()
            
            out = net(x)        
  
            loss = loss_func(out, y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
            progress.set_description("loss: %f.3" % np.mean(train_loss) )
            del(x)
            del(y)
            del(out)
            del(loss)

        train_losses.append(np.mean(train_loss))
        plt.title("epoch:{} train loss: {},\n".format(epoch, train_losses[-1]))
        display.clear_output(wait=True)
     
        torch.save(net.state_dict(), folderToDump+"/" + str(epoch))
        logs = dict()
        logs["trainLoss"] = train_loss
        
        with open( folderToDump + "/log" + str(epoch) + ".txt", "w" ) as f:
          json.dump(logs, f)

In [ ]:
train(data_loader, model, optimizer, loss_function, epoch_num)

In [ ]:
model.load_state_dict(torch.load(folderToDump+"/" + str(4)))

In [ ]:
featureExtractor = nn.Sequential(*list(model.children())[:-1]).cuda()

In [ ]:
featureExtractor

In [ ]:
carLabelMap = dict()
count = 0
for carClass in dataset.classesMap.values():
  if carClass not in carLabelMap:
    carLabelMap[carClass] = count
    count += 1
for carClass in dataset2.classesMap.values():
  if carClass not in carLabelMap:
    carLabelMap[carClass] = count
    count += 1
      
meanFeatureVectors = np.zeros((len(carLabelMap.keys()), 2048))
vectorCounts = np.zeros((len(carLabelMap.keys())))

In [ ]:
featureExtractor.eval()
featureVectors = []
paths = []
labels = []
for x, y, path in tn(data_loader):
  out = featureExtractor(x.cuda()).detach().cpu().numpy()
  featureVectors.extend(list(out))
  labels.extend(y.numpy())
  paths.extend(path)

  for index in range(len(path)):
    meanFeatureVectors[carLabelMap[y[index].item()]] += out[index]
    vectorCounts[carLabelMap[y[index].item()]] += 1




In [ ]:
featureExtractor.eval()

for x, y, path in tn(data_loader2):
  out = featureExtractor(x.cuda()).detach().cpu().numpy()
  featureVectors.extend(list(out))
  labels.extend(y.numpy())
  paths.extend(path)

  for index in range(len(path)):
    meanFeatureVectors[carLabelMap[y[index].item()]] += out[index]
    vectorCounts[carLabelMap[y[index].item()]] += 1




In [ ]:
data = dict()
data["features"] = featureVectors
data["path"] =paths
data["labels"] = labels
data["meanFeatureVectors/"] = meanFeatureVectors
data["vectorCounts"] = vectorCounts

In [ ]:
import pickle

In [ ]:
with open('drive/My Drive/data.pickle', 'wb') as f:
  pickle.dump(data, f)